# Importing Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
train = pd.read_csv('/Users/joohyun/Desktop/Projects/bike-sharing-demand/train.csv', parse_dates=['datetime'])
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv('/Users/joohyun/Desktop/Projects/bike-sharing-demand/test.csv', parse_dates=['datetime'])
print(test.shape)
test.head()

# Modeling - linear regression

## Preprocessing

steps for train data
1. Create necessary columns from datetime: year, month, hour, dayofweek
2. Remove rows where weather has value 4 (outliers)
3. Remove outliers from the count column: values exceeding 3 standard deviations
4. Apply log transformation to the target columns: casual and registered
5. Convert categorical variable types
6. Split the data into 2 datasets (one for casual, one for registered)

#### 1. Creating New Columns

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek
del train['datetime']

train.head(3)

#### 2. Removing the Outlier in `weather` Column

In [ ]:
print('before removing:', len(train))
train = train.loc[train['weather'] !=4 ]
print('after removing: ', len(train))

#### 3. Removing the Outliers in `count` Column

anything that are greater than 3σ = outlier

In [ ]:
train = train[train['count'] - train['count'].mean() < 3*train['count'].std()]
train.reset_index(inplace =True, drop = True)
print(train.shape)
display(train.head())

#### 4. log transformation of `casual` & `registered` columns

In [ ]:
# distributions of casual & registered
figure, (ax1, ax2) = plt.subplots(1,2, figsize = (10,5))
sns.histplot(train['casual'], ax = ax1, kde=True)
sns.histplot(train['registered'], ax = ax2, kde=True)
plt.show()

In [ ]:
# changing to log
figure, (ax1, ax2) = plt.subplots(1,2, figsize = (10,5))
sns.histplot(np.log1p(train['casual']), ax = ax1, kde=True)
sns.histplot(np.log1p(train['registered']), ax = ax2, kde=True)

plt.show()

In [ ]:
train['casual'] = np.log1p(train['casual'])
train['registered'] = np.log1p(train['registered'])

display(train.head())


In [ ]:
print(train['casual'].skew())
print(train['registered'].skew())

#### 5. Changing the types of categorical column

In [ ]:
category_name = ['weather', 'season', 'year', 'month', 'hour', 'dayofweek']

for c in category_name:
    train[c] = train[c].astype('category')
train.dtypes

In [ ]:
train.drop(['count'], axis = 1,inplace =True)

#### 6. Spliting into 2 DFs

1. `casual`
2. `registered`

In [ ]:
train_c =  train.drop(['registered'], axis = 1)
train_re = train.drop(['casual'], axis = 1)

print(train_c.shape, train_re.shape)
display(train_c.head(3), train_re.head(3))

#### 7. Setting ML Prediction for `casual` and `registered`

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
X_c = train_c.drop(['casual'], axis = 1) 
y_c = train_c['casual']

In [ ]:
X_r = train_re.drop(['registered'], axis = 1) 
y_r = train_re['registered']

## `casual` Prediction

#### Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

#linear regression model
lr_reg = LinearRegression()

#Performed feature standardization together with k-fold cross-validation
pipe = make_pipeline(StandardScaler(), lr_reg)
scores = cross_validate(pipe, X_c, y_c, cv=5, scoring='neg_mean_squared_error',return_train_score=True)

In [ ]:
print("MSLE: {0:.3f}".format(np.mean(-scores['test_score'])))

In [ ]:
np.sqrt(0.933)

#### MinMax Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#linear regression model
lr_reg = LinearRegression()

#Performed feature standardization together with k-fold cross-validation
pipe = make_pipeline(MinMaxScaler(), lr_reg)
scores = cross_validate(pipe, X_c, y_c, cv=5, scoring='neg_mean_squared_error',return_train_score=True)

In [ ]:
print("RMSLE: {0:.3f}".format(np.sqrt(np.mean(-scores['test_score']))))

In [ ]:
np.sqrt(0.966)

## `registered` prediction

#### MinMax Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#linear regression model
lr_reg = LinearRegression()
#Performed feature standardization together with k-fold cross-validation
pipe = make_pipeline(MinMaxScaler(), lr_reg)
scores = cross_validate(pipe, X_r, y_r, cv=5, scoring='neg_mean_squared_error',return_train_score=True)

In [ ]:
print("RMSLE: {0:.3f}".format(np.sqrt(np.mean(-scores['test_score']))))

In [ ]:
np.sqrt(1.031)

# Modeling - Ridge (`casual`)

### Robust Scaler

In [ ]:
from sklearn.preprocessing import RobustScaler

#Ridge
ridge_reg = Ridge()

In [ ]:
#Tuned the alpha parameter in Ridge regression.
pipeline = Pipeline([('scaler', RobustScaler()), ('ridge',Ridge())])
params={'ridge__alpha':[5, 10, 15, 20]}
grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5)
grid_model.fit(X_c, y_c)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

In [ ]:
np.sqrt(0.924)

### MinMax Scaler

In [ ]:
from sklearn.preprocessing import RobustScaler

#Ridge
ridge_reg = Ridge()

In [ ]:
#Tuned the alpha parameter in Ridge regression.
pipeline = Pipeline([('scaler', MinMaxScaler()), ('ridge',Ridge())])
params={'ridge__alpha':[5, 10, 15, 20]}
grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5)
grid_model.fit(X_c, y_c)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

In [ ]:
np.sqrt(0.924)

# Modeling - Random Forest (`casual`) - MinMax Scaler

In [ ]:
np.random.seed(0)
rf =  RandomForestRegressor(n_estimators=500)

In [ ]:
#Performed feature standardization together with k-fold cross-validation
pipe = make_pipeline(MinMaxScaler(), rf)
scores = cross_validate(pipe, X_c, y_c, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

In [ ]:
print("MSLE: {0:.3f}".format(np.mean(-scores['test_score'])))

In [ ]:
#Tuned the hyperparameter in Random Forest.
pipeline = Pipeline([('scaler', MinMaxScaler()), ('rf',RandomForestRegressor())])

params={'rf__max_depth': [5,10,None],
       'rf__min_samples_leaf': [1,3],
       'rf__min_samples_split': [2, 3],
        'rf__n_estimators': [500, 1000]}

grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5, n_jobs = 5, verbose=True)
grid_model.fit(X_c, y_c)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

# Modeling - LGBM - MinMax Scaler

## `casual` Prediction

In [ ]:
#LGBM
lgbm = LGBMRegressor(n_estimators = 500, objective = 'regression')

In [ ]:
#Performed feature standardization together with k-fold cross-validation
pipe = make_pipeline(MinMaxScaler(), lgbm)
scores = cross_validate(pipe, X_c, y_c, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
print("MSLE: {0:.3f}".format(np.mean(-scores['test_score'])))

In [ ]:
#Tuned the hyperparameter in LGBM.
pipeline = Pipeline([('scaler', MinMaxScaler()), ('lgbm',LGBMRegressor(objective='regression'))])

params={'lgbm__learning_rate': [0.001, 0.01, 0.1],
       'lgbm__max_depth': [5, 10],
       'lgbm__reg_lambda':[0.1, 1],
       'lgbm__subsample': [0.5, 1],
       'lgbm__n_estimators': [500, 1000]}

grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5, n_jobs = 5, verbose=True)
grid_model.fit(X_c, y_c)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

In [ ]:
#Tuned the hyperparameter again.
pipeline = Pipeline([('scaler', MinMaxScaler()), ('lgbm',LGBMRegressor(objective='regression', learning_rate = 0.01, subsample = 0.5))])

params={'lgbm__max_depth': [10, 15],
       'lgbm__reg_lambda':[0.1, 1],
       'lgbm__n_estimators': [500, 1000]}

grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5, n_jobs = 5, verbose=True)
grid_model.fit(X_c, y_c)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

## `registered` Prediction

In [ ]:
# X, y 나누기
X_r = train_re.drop(['registered'], axis = 1) 
y_r = train_re['registered']

In [ ]:
#LGBM
lgbm = LGBMRegressor(n_estimators = 500, objective = 'regression')

#Performed feature standardization together with k-fold cross-validation.
pipe = make_pipeline(MinMaxScaler(), lgbm)
scores = cross_validate(pipe, X_r, y_r, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

In [ ]:
print("MSLE: {0:.3f}".format(np.mean(-scores['test_score'])))

In [ ]:
#Tuned the hyperparameter in LGBM.
pipeline = Pipeline([('scaler', MinMaxScaler()), ('lgbm',LGBMRegressor(objective='regression', learning_rate = 0.1, subsample = 0.5))])
params={'lgbm__max_depth': [3, 5, 7],
       'lgbm__reg_lambda':[0.1, 1],
       'lgbm__n_estimators': [300, 500]}
grid_model = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_squared_error', cv=5, n_jobs = 5, verbose=True)
grid_model.fit(X_r, y_r)

In [ ]:
print("MSLE: {0:.3f}".format( -1*grid_model.best_score_)) 
print('optimal hyperparameter: ', grid_model.best_params_)

# Test the Data

## final parameter
LGBMRegressor(n_estimators = 500, objective = 'regression', learning_rate = 0.1, max_depth = 3), reg_lambda = 0.1, subsample = 0.5)

## Preprocessing

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek
del test['datetime']

In [ ]:
test.head()

In [ ]:
category_name = ['weather', 'season', 'year', 'month', 'hour', 'dayofweek']

for c in category_name:
    test[c] = test[c].astype('category')
test.dtypes

## `casual` Prediction

In [ ]:
# feature standardization
minmax = MinMaxScaler()
minmax.fit(X_c)
X_c_scaled = minmax.transform(X_c)
X_test_scaled = minmax.transform(test)

In [ ]:
# Training with the final parameter-tuned model
lgbm1 = LGBMRegressor(n_estimators = 500, objective = 'regression',
                    learning_rate = 0.01, subsample = 0.5, max_depth = 15, reg_lambda = 0.1, ranodm_state = 99)

In [ ]:
# Learning
lgbm1.fit(X_c_scaled, y_c)

In [ ]:
# Make predictions on the test set.
pred_c = lgbm1.predict(X_test_scaled)
fpred_c = np.expm1(pred_c)

In [ ]:
#lgbm1's feature importance
imp_casual = pd.DataFrame({'feature': test.columns, 
                                    'coefficient': lgbm1.feature_importances_})
imp_casual = imp_casual.sort_values(by = 'coefficient', ascending = False)

plt.barh(imp_casual['feature'], imp_casual['coefficient'])
plt.show()

- According to the EDA results, hour was identified as the most important variable, and the LGBM model also assigned it a very high level of importance.
- Next in importance were: day of the week > humidity > temperature > month > year.

## `registered` Prediction

In [ ]:
# feature importance
minmax = MinMaxScaler()
minmax.fit(X_r) 
X_r_scaled = minmax.transform(X_r)
X_test_scaled = minmax.transform(test)

In [ ]:
# Training with the final parameter-tuned model
lgbm2 = LGBMRegressor(n_estimators = 500, objective = 'regression',learning_rate = 0.1, max_depth = 3, reg_lambda = 0.1, subsample = 0.5, random_state = 99)

In [ ]:
# learning
lgbm2.fit(X_r_scaled, y_r)

In [ ]:
# Make predictions on the test set.
pred_re = lgbm2.predict(X_test_scaled)
fpred_re = np.expm1(pred_re) 

In [ ]:
#lgbm2's feature importance
imp_re = pd.DataFrame({'feature': test.columns, 
                                    'coefficient': lgbm2.feature_importances_})
imp_re = imp_re.sort_values(by = 'coefficient', ascending = False)

plt.barh(imp_re['feature'], imp_re['coefficient'])
plt.show()

- In the registered model as well, hour had the highest contribution, followed by humidity > day of the week > temperature > working day.